## Stage 0
Extracting data from https://data.cityofnewyork.us/Public-Safety/NYPD-Motor-Vehicle-Collisions-Crashes/h9gi-nx95 and filtering by borough and cyclists who were killed or injured.

In [1]:
import requests
boroughs = ['BRONX', 'QUEENS', 'STATEN ISLAND', 'BROOKLYN', 'MANHATTAN']
borough = boroughs[-1]
url = 'https://data.cityofnewyork.us/resource/h9gi-nx95.json?' + \
      '$$app_token=HDCRkI7kFAjtZfZUc9ww2WCpM&' + \
      f'$where=borough="{borough}" AND' + \
      '(number_of_cyclist_injured>0 OR number_of_cyclist_killed>0) AND' + \
      '(crash_date between "2019-06-01T12:00:00" and "2019-09-01T12:00:00")'

req = requests.get(url)
cycle_data = eval(req.text)

In [2]:
len(cycle_data)

335

In [3]:
import datetime as dt

for c in cycle_data:
    time = c['crash_date'].split('T')[0] + 'T' + c['crash_time']
    time_of_crash = dt.datetime.strptime(time, '%Y-%m-%dT%H:%M')
    c['crash_datetime'] = time_of_crash

## Stage 1
Store the data from Stage 0 in a database using a proper data model

In [4]:
import sqlite3

def connect_db():
    try:
        conn = sqlite3.connect('./data/database.db', detect_types=sqlite3.PARSE_DECLTYPES)
    except sqlite3.Error as err:
        print(err)
    if conn:
        return conn

In [5]:
conn = connect_db()
cursor = conn.cursor()
create_table_sql = '''
CREATE TABLE IF NOT EXISTS bike_stations(
 id INTEGER PRIMARY KEY,
 latitude DOUBLE,
 longitude DOUBLE
)
'''
cursor.execute(create_table_sql)
conn.commit()

In [6]:
for c in cycle_data:
    try:
        insert_cycle_data_sql = f'''
        INSERT OR IGNORE INTO collisions(
                               collision_id, 
                               crash_datetime, 
                               latitude, 
                               longitude, 
                               cyclist_injured, 
                               cyclist_killed,
                               borough)
            VALUES('{c['collision_id']}', 
                   '{c['crash_datetime']}', 
                   '{c['latitude']}', 
                   '{c['longitude']}',
                   '{c['number_of_cyclist_injured']}',
                   '{c['number_of_cyclist_killed']}',
                   '{c['borough']}')
        '''
        cursor.execute(insert_cycle_data_sql)
    except KeyError:
        # don't include data points that don't have locations
        continue
        
conn.commit()
conn.close()

## Stage 2
Visualize in the data in a map and put it in a website

In [7]:
import pandas as pd

conn = connect_db()
select_data_sql = f"SELECT * FROM collisions WHERE borough='{borough}'"
df_collisions = pd.read_sql_query(select_data_sql, conn)
conn.close()

In [8]:
conn = connect_db()
select_data_sql = f"SELECT * FROM bike_stations"
df_stations = pd.read_sql_query(select_data_sql, conn)
conn.close()

In [43]:
conn = connect_db()
tmp = pd.read_sql_query('SELECT DISTINCT borough FROM collisions', conn)
tmp = tmp.iloc[:,0].tolist()
conn.close()
tmp

['QUEENS', 'STATEN ISLAND']

In [13]:
from haversine import haversine
def get_close_collisions(station_lat, station_lon, df_collisions):
    num_close = 0
    for _, c in df_collisions.iterrows():
        distance = haversine((station_lon, station_lat), (c['longitude'], c['latitude']))
        if distance < CLOSE_DISTANCE / 1000: 
            num_close += 1
    return num_close

df_stations['close_collisions'] = df_stations.apply(lambda r: get_close_collisions(r['latitude'], r['longitude'], df_collisions), axis=1)


In [32]:
import folium
from folium.plugins import MarkerCluster

def get_color(num_collisions, quartiles):
    if num_collisions <= quartiles[1]:
        return 'green'
    elif num_collisions > quartiles[1] and num_collisions < quartiles[2]:
        return '#f59b42'
    else:
        return 'red'

geo_map = folium.Map(location=[40.74527, -73.988573], tiles=None, zoom_start=12)

# collisions
folium.TileLayer('Stamen Terrain', name='Collision Data').add_to(geo_map)
for borough in boroughs:
    feature_group = folium.FeatureGroup(name=borough)
    marker_cluster = MarkerCluster().add_to(feature_group)
    df_collisions[df_collisions['borough'] == borough].apply(
        lambda r: folium.Marker(
            location=[r['latitude'], r['longitude']],
            tooltip=f"{r['cyclist_injured'] + r['cyclist_killed']} cyclists injured/killed",
            icon=folium.Icon(color='darkblue', icon='exclamation-circle', prefix='fa')).add_to(marker_cluster), axis=1)
    feature_group.add_to(geo_map)

# bike stations
quartiles = df_stations['close_collisions'].quantile([0.25,0.5,0.75]).tolist()
feature_group = folium.FeatureGroup(name='Bike Stations')
df_stations.apply(lambda r: folium.CircleMarker(
                      location=[r['latitude'], r['longitude']], 
                      radius=2,
                      color=get_color(r['close_collisions'], quartiles),
                      tooltip=f"{int(r['close_collisions'])} collisions within {CLOSE_DISTANCE}m",
                      fill=True).add_to(feature_group), axis=1)
feature_group.add_to(geo_map)

folium.LayerControl(collapsed=False).add_to(geo_map)
geo_map